###Super módulo de Node, aula 7

Na aula passada vimos um pouco sobre o ORM Prisma, vimos o poder de criar tabelas com tipos específicos SQL com o Prisma, vimos alguns tipos de relacionamentos que o Prisma é capaz de fazer e executamos algumas query's básicas.

Na aula de hoje, daremos continuidade ao aprendizado, porém, agora, iremos introduzir o conceito do TypeScript.

###Por que utilizar o TypeScript?

TypeScript é uma linguagem de programação que estende o JavaScript, adicionando tipagem estática e outros recursos que tornam o desenvolvimento mais robusto e produtivo. Quando se trata de criar APIs com Node.js e Express, utilizar TypeScript traz diversos benefícios:

**1. Segurança e Confiabilidade**

Com a tipagem estática do TypeScript, você define os tipos de dados esperados para variáveis, parâmetros e retornos de funções. Isso ajuda a identificar erros durante o desenvolvimento, antes mesmo da execução.


Exemplo:
```
function getUser(id: number): string {
    return `User ID: ${id}`;
}

// Erro detectado no editor:
getUser("abc"); // Argumento inválido!
```

Isso reduz a chance de bugs causados por tipos inesperados em tempo de execução, o que é especialmente importante em APIs que lidam com validação e manipulação de dados.

**2. Melhor Experiência de Desenvolvimento**

TypeScript oferece suporte completo para ferramentas modernas, como o IntelliSense, que ajuda com:

    Sugestões automáticas.
    Validação de código em tempo real.
    Refatoração fácil.

Essa experiência melhora a produtividade e reduz erros.


**3. Código Mais Legível e Manutenível**

Com TypeScript, você pode usar interfaces e tipos para documentar contratos claros entre diferentes partes do código. Isso torna o código mais fácil de entender e ajuda novos desenvolvedores a se adaptarem rapidamente ao projeto.

Exemplo:


```
interface User {
    id: number;
    name: string;
}

function createUser(user: User): string {
    return `User ${user.name} created with ID ${user.id}`;
}

```
**4. Suporte a Recursos Modernos do JavaScript**

TypeScript é compatível com os recursos mais recentes do JavaScript e permite que você use sintaxes modernas com segurança. Além disso, ele transpila o código para versões mais antigas do JavaScript, garantindo compatibilidade com diferentes ambientes.


**5. Escalabilidade**

Em projetos maiores, a ausência de tipagem no JavaScript pode levar a problemas de manutenção e inconsistências no código. TypeScript ajuda a:

    Garantir que as mudanças em um módulo não quebrem outros módulos.
    Oferecer uma base sólida para projetos que crescem com o tempo.


**6. Ecossistema Amigável**

A maioria das bibliotecas e frameworks, incluindo Express, já tem tipos disponíveis através do @types, permitindo integração fácil e tipagem pronta para uso.

Exemplo:

```
import express, { Request, Response } from 'express';

const app = express();

app.get('/users', (req: Request, res: Response) => {
    res.send('Hello, TypeScript!');
});
```



##Projeto

Certo, agora que vimos as vantagens do TypeScript, vamos construir um pequeno projeto com Express, prisma e TypeScript, para compreendermos melhor essas estruturas

Para começarmos o nosso projeto, vamos fazer alguns passos:

Primeiro: Vamos criar a pasta do nosso projeto

Segundo: Vamos instalar as nossas ferramentas para esse projeto



```
npm install express
npm install @prisma/client
npm install @types/express@^4.17.17 -D
npm install @types/node -D
npm install eslint -D
npm install @typescript-eslint/eslint-plugin -D
npm install @typescript-eslint/parser -D
npm install typescript -D
npm install ts-node-dev -D
npm install prisma -D
```

Terceiro: Agora, precisamos inicializar o nosso typeScript

```
npx tsc --init
```

Quarto: Agora, inicializado o TS, temos aí um arquivo tsconfig.json, nesse arquivo vamos fazer uma pequena alteração, no Strict, vamos colocar ele como false

Quinto: Vamos inicializar o eslint

```
npx eslint --init
```

Agora, fizemos todas as alterações básicas para a construção do nosso projeto

Agora, vamos criar uma pasta chamada src, nessa pasta, criaremos o arquivo server.ts, onde colocaremos o seguinte código

```
import Express from "express";

const app = Express()
app.use(Express.json())

const PORT = 8000

app.get('/', (request, response) => {
    return response.send({ message: 'Hello World' })
})

app.listen(PORT, () => {
    console.log(`O servidor está rodando ${PORT}`);
})
```

Pronto, com esse código agora, teremos a disponibilidade de uma porta e de criarmos as nossas rotas.

Com isso, agora, devemos inicializar o prisma:

```
npx prisma init --datasource-provider sqlite
```

Inicializado o nosso prisma, vamos agora construir o nosso modelo de usuário

```
// This is your Prisma schema file,
// learn more about it in the docs: https://pris.ly/d/prisma-schema

generator client {
  provider = "prisma-client-js"
}

datasource db {
  provider = "sqlite"
  url      = env("DATABASE_URL")
}

model User {
  id Int @id @default(autoincrement())
  name String
  email String @unique

  @@map("users")
}

```

A função map, serve basicamente para modificar o nome da nossa entidade, ou seja, ela não pega assim o valor diretamente da nossa model

Agora, vamos criar uma tabela de posts, que vai ser relacionada com os Posts que o usuário pode fazer.

```
model Post {
  id Int @id @default(autoincrement())
  title String?
  content String
  userId Int
  autor User @relation(fields: [userId], references: [id])
}
```

Agora, criado os nosso modelos para essa aplicação, agora, vamos criar as nossas funcionalidades, endpoint's das nossas API's, para isso, vamos criar agora, duas pastas, a primeira chamada *database* e a segunda chamada *controllers*, cada módulo desse terá sua responsabilidade dentro da nossa aplicação, vamos começar pelo database, nessa pasta, vamos criar um arquivo chamado index.ts, nele teremos a intância do nosso ORM Prisma



```
import { PrismaClient } from "@prisma/client";

const prisma = new PrismaClient()

export { prisma }
```

Agora, dentro da nossa pasta controllers, vão ficar as nossas lógicas para gerenciar a nossa aplicação, com verificações se o usuário já existe, com inserção de dados e tudo mais.

```
import { Request, Response } from 'express';
import { prisma } from '../database';

const UserController = {
    async createUser(request: Request, response: Response): Promise<Response> {
        try {
            const { name, email } = request.body;
            const userExists = await prisma.user.findUnique({ where: { email } });

            if (userExists) {
                return response.json({
                    error: true,
                    message: 'Erro: Usuário já existe'
                });
            }

            const user = await prisma.user.create({
                data: {
                    name,
                    email
                }
            });

            return response.json({
                error: false,
                message: 'Sucesso: Usuário cadastrado com sucesso',
                user
            });
        } catch (error) {
            return response.json({ message: error.message });
        }
    }
};

export default UserController;

```

Criado o controller de user, agora podemos chamar no nosso server a nossa rota para cadastrar usuário

app.post('/createUser', UserController.createUser)

Agora, podemos utilizar o insomnia, o postman ou thunderclient para cadastrar o nosso usuário no nosso banco de dados

Continuando o assunto, vamos agora cadastrar os posts do usuário, esses posts serão feitos da mesma forma que o controller de usuário

AGora é com você, tente fazer o controller de Posts com o GET, POTS, PUT e DELETE.Caso queira dar uma olhada no código completo dessa aula, visite o esse repositório https://github.com/FelipeHardmann/api_express_ts